# Introduction

This notebook is used to test and illustrate the v0.1 of the DSGE model library.

At this stage, the functionalities of the library are limited. Namely two important features are missing for economic modeling purpose:
* A lag operator to declare variables such as $x_t = f(x_{t-1})$, which prohibits modeling time dependant variables and shocks
* Indexing so a variable that has been defined can be reused multiple times in different contexts

Therefore, I provide here a very limited example, mostly desgined to test the parser.

# The Model - A Centralized Economy

## Constrained Opitmization

We assume that the economy is administred by a central planner whose aim is to maximize the utility of households. We use a representative household for the optimization of utility. In addition, we assume that the quantity of labor is fixed at each period.

Our economy is constrained by the national income identity, change in stock of capital and its production functions:
$$
\begin{cases}
y_t = c_t + i_t \\
\Delta k_{t+1} = i_t - \delta k_t \\
y_t = F(k_t)
\end{cases}
$$

We can rewrite these equations as a single constraint on the economy:
$$ F(k_t) = c_t + \Delta k_{t+1} + \delta k_t $$

Therefore, by denoting the utility function by $U(c_t)$ and the discount factor $\beta = \frac{1}{1+\theta}$ where $\theta$ is the discount factor of the representative household (assumed constant), the central planner aims at maximizing:

$$
\begin{aligned}
& \underset{\{c_{t+s},k_{t+s}|s\in N\}}{\text{min}}
& & V_t = \sum_{s=0}^\infty{\beta^sU(c_{t+s})} \\
& \text{s.t.}
& & F(k_t) = c_t + \Delta k_{t+1} + \delta k_t
\end{aligned}
$$

The lagrangian for this optimization yields:
$$
\newcommand{\lagr}{\mathcal{L}}
\lagr_t = \sum_{s=0}^\infty[\beta^sU(c_{t+s})+\lambda_{t+s}(F(k_{t+s}) - c_{t+s} - k_{t+s+1}+(1-\delta)k_{t+s})]
$$

The derivation for $s\in N$ yields:
$$
\begin{cases}
\frac{\partial\lagr_t}{\partial c_{t+s}} = \beta^sU'(c_{t+s})-\lambda_{t+s} \\
\frac{\partial\lagr_t}{\partial k_{t+s}} = \lambda_{t+s}(F'(k_{t+s})+1-\delta)-\lambda_{t+s-1}
\end{cases}
$$

Combining these equations gives us the Euler equation:
$$
\beta\frac{U'(c_{t+1})}{U'(c_t)}(F'(k_{t+1})+1-\delta)=1
$$

## Application: Isoelastic Utility and Cobb-Douglas Production Function

For the remainder, we assume that the utility function follows the isoelastic utility function and the production function is Cobb-Douglas:
$$
\begin{cases}
U(c) = \frac{c^{1-\sigma}-1}{1-\sigma} \\
F(k) = A k^\alpha
\end{cases}
$$

At equilibrium, $c_t = c_{t+1} = c^*$ and $k_t = k_{t+1} = k^*$, therefore the Euler equation can be rewritten as:
$$
A\alpha {k^*}^{\alpha-1}+1-\delta = \frac{1}{\beta}=1+\theta
$$

Rearranging the terms gives:
$$
{k^*}={\left(\frac{A\alpha}{\delta+\theta}\right)}^{\frac{1}{1-\alpha}}
$$

And using this result in $c^* = F(k^*)-\delta k^*$ yields:
$$
c^* = k^*\left(\frac{\delta(1-\alpha)+\theta}{\alpha}\right)
$$

We will now simulate the following system using the library.
$$
\begin{cases}
k = {\left(\frac{A\alpha}{\delta+\theta}\right)}^{\frac{1}{1-\alpha}} \\
c = k\left(\frac{\delta(1-\alpha)+\theta}{\alpha}\right) \\
y = c + \delta k_t
\end{cases}
$$

In addition, we will add $F=A k^\alpha$, which should equal $y$.


# Implementation of the Model

## Instantiation of the Model

The library provides a class named Econ_model which allows parsing a system, reading parameter values and running the model. With the current version (v0.1), an Econ_model class needs to be instantiated with file path to the system and variables. This will be changed in future versions.

In [15]:
from DSGE.Econ_model import Econ_model

from os import getcwd
from os.path import join

import pandas as pd

The description of the system is a plain text file, in which each line should have the following form:

>variable name (alphanumeric) = function of variables and parameters

for example:

> Y = A*X + B

The Econ_model instance will then parse each line before running the model. In the case of the model described above, the content of the file is:

In [34]:
var_file = join(getcwd(),'models/V0.1/variables_v0.1')
with open(var_file,'r') as f:
    for line in f:
        print(line)

k = ((A*alpha)/(delta+theta))**(1/(1-alpha))

c = k*(delta*(1-alpha)+theta)/alpha

y = c + delta * k

F = A * k ** alpha



The file containing the parameters is a simple json. Currently, the Econ_model instance does not check whether all parameters are present or not.

The required parameters for our model are $A$, $\alpha$, $\theta$ and $\delta$. I assigned them a value for the purpose of the example as described below:

In [39]:
param_file = join(getcwd(),'models/V0.1/parameters_v0.1')
with open(var_file,'r') as f:
    for line in f:
        print(line)

k = ((A*alpha)/(delta+theta))**(1/(1-alpha))

c = k*(delta*(1-alpha)+theta)/alpha

y = c + delta * k

F = A * k ** alpha



With these files, we can now instantiate our model. An extra argument required is the name of the model, 'Model V0.1' in this case.

We can run the model with the following line:

> model(1,1)

The first parameter refers to the number of simulations while the second is the number of iteration. Since the model is entirely deterministic and assumes a long-term equilibrium, only one iteration is required.

In [40]:
# Instanciation of the model
model = Econ_model(
    'Model V0.1', # Name of the model
    var_file,     # File containing the definitions of the variables
    param_file    # File containing the value of each parameters
)

# Running the simulation
model(
    1,    # Number of simulations
    1     # Number of iterations
)

We can now retrive the results stored in the 'results' attribute of the Econ_model instance.

Currently, 'results' stores all variables and parameters for each simulation and each iterations in a dictionary. This will be changed later to allow more flexibility and improve the performance of the library.

We display below the simulation results:

In [41]:
tmp_results = {n:val[0] for n,val in model.results.items()}
pd.DataFrame(index=['results'],data=tmp_results)

,theta,alpha,A,delta,k,c,y,F
results,0.05,0.3,1,0.12,2.25107,1.005478,1.275606,1.275606


We see in the table above the values that were attributed in the parameters file for $A$, $\alpha$, $\theta$ and $\delta$. The endogeneous variables $k$, $c$, $y$ and $F$ are also displayed.

As expected, we have the equality $y=F$.

# Conclusion

This notebook demonstrates the sound functioning of the Econ_model class. The library currently has limited capabilities, but future versions should make it more functional.

Prospective steps are:
* v0.2: Addition of the lag operator
* v0.3: Addition of an indexing system
* v1.0: Improvements to obtain a fully functional version